#### **Author: Wissem Khlifi**

# 01 - BigQuery - Table Data Source
Use BigQuery to load and prepare data for machine learning. This includes understanding the data source,
feature selection, and splitting datasets into testing, validation, and training datasets.



## Source Data
Specify the BigQuery table that will be used as the data source.
This table contains the dataset for the machine learning project.
BigQuery Table: `<Your Project ID>.ml_datasets.ulb_fraud_detection`

### Explanation of Data Splitting

Splitting data into training, validation, and testing sets is crucial in machine learning to ensure that
the model generalizes well to new, unseen data. It helps in evaluating the model's performance and tuning it effectively.

- Training Data: This is the portion of the dataset used to train the model. The model learns patterns and relationships
in the training data to make predictions. It typically comprises the largest portion of the dataset.

- Validation Data: This subset is used to tune the model's hyperparameters and to provide an unbiased evaluation
of the model during training. It helps in selecting the best model architecture and avoiding overfitting.

- Testing Data: The test set is used to evaluate the final model's performance. It provides an unbiased measure
of the model's accuracy and generalization to new data, as it is not used during the training or validation phases.



## Setup

### inputs

Get the GCP project ID from the gcloud configuration
This ensures the notebook uses the correct GCP project for all operations.


In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'genai-demo-2024'

### Define variables for the rest of the Notebook


In [8]:
# Specifying the region is necessary for certain GCP services to function correctly.
REGION = 'us-central1'
# Define experiment series : This helps in organizing and tracking different experiment runs.
EXPERIMENT = '01'
SERIES = '01'

# Define source data parameters
# These variables specify the BigQuery dataset and table to be used.
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'ml_datasets'
BQ_TABLE = 'ulb_fraud_detection'

# Construct the full BigQuery source path
# This combines project ID, dataset, and table into a single reference for BigQuery operations

BQ_SOURCE = f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}'
BQ_SOURCE


'genai-demo-2024.ml_datasets.ulb_fraud_detection'

### Import necessary Packages:
These imports are required for interacting with BigQuery, Google Cloud Storage, and Google Cloud Authentication.


In [7]:
from google.cloud import bigquery
from google.cloud import storage
from google.auth import compute_engine, default
import google.auth
import google.auth.transport.requests

### Clients
Initialize clients for GCP services
Creating client instances allows interaction with GCP services within the notebook.


In [9]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

### Parameters
Set parameters
Using the project ID as the bucket name for simplicity and to ensure uniqueness.


In [10]:
BUCKET = PROJECT_ID

### Retrieve and Review a Sample From The Table:
This section demonstrates how to query a sample of data from the BigQuery table to review it before further processing.

> **Note:** The `LIMIT 5` statement does limit the number of rows returned by BigQuery to 5 but BigQuery still does a full table scan.  
If you have a table larger than 1GB and want to limit the rows scanned for a quick review like then then replacing `LIMIT 5` with `TABLESAMPLE SYSTEM (1 PERCENT)` would be more efficient.  For tables under 1GB it will still return the full table.  
More on [Table Sampling](https://cloud.google.com/bigquery/docs/table-sampling)

In [13]:
# Construct a query to retrieve a sample of data from the BigQuery table
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
# Execute the query and convert the result to a pandas dataframe for easy manipulation and review in Python
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Feedback
0,149835.0,-0.208098,0.567463,-0.811635,-1.831501,0.442591,-0.236864,0.200599,0.345577,-1.709567,...,0.575520,-0.011990,0.138785,-0.914909,0.578927,0.319829,0.233426,8.00,0,very satisfied.
1,26815.0,-0.420297,0.483617,0.868779,-2.045578,0.452421,-0.324837,0.618529,-0.135828,-1.580132,...,-0.502885,-0.089659,-0.784473,-0.139855,-0.644991,-0.037222,0.100003,8.00,0,very satisfied.
2,81676.0,1.104733,-0.185207,0.439561,1.182593,0.079340,1.294311,-0.455057,0.370983,0.643557,...,0.035177,-0.255415,-1.103956,0.739641,-0.207123,0.064421,0.006113,36.00,0,very satisfied.
3,94019.0,2.005148,0.408539,-1.815424,1.458494,0.742022,-1.286774,0.788253,-0.577383,1.062030,...,0.424273,0.003339,1.093558,0.495325,-0.515839,-0.075583,-0.057191,36.00,0,very satisfied.
4,94032.0,2.025415,0.471448,-1.811719,1.462872,0.767903,-1.443030,0.897959,-0.697742,0.967117,...,0.476975,-0.095885,-0.051129,0.563746,-0.458306,-0.065255,-0.066606,36.00,0,very satisfied.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210315,77700.0,1.219388,0.560330,-0.475506,0.812431,0.006552,-1.267116,0.264263,-0.144057,-0.334000,...,-0.293571,-0.062173,0.404417,0.489581,0.341182,-0.038846,0.033249,0.76,0,Very happy with the quick and efficient service.
210316,78256.0,-0.714316,1.402217,0.018498,0.083353,0.630271,-0.181795,0.400530,0.656264,-1.006304,...,0.209476,-0.062232,-0.431059,-0.519844,0.302932,0.013695,0.103502,0.76,0,Very happy with the quick and efficient service.
210317,82025.0,-0.580180,-0.314571,2.041901,-2.982935,-0.195448,-0.246573,0.092692,0.088708,2.046032,...,0.464865,-0.196856,0.042678,-0.118104,-1.044651,0.053008,-0.089352,0.76,0,Very happy with the quick and efficient service.
210318,136310.0,-1.102492,2.159172,-4.312176,-1.163056,3.300516,2.900514,-0.881430,-1.150740,-1.261079,...,0.552558,0.064139,0.503571,-0.209685,0.579692,-0.335457,0.021402,0.76,0,Very happy with the quick and efficient service.


### [OPTIONAL] Check out this table in BigQuery Console:
Provides a direct link to the BigQuery console for further exploration and verification of the dataset.
This helps to manually inspect and validate the data directly in BigQuery's interface.

- Click: https://console.cloud.google.com/bigquery
- Make sure project selected is the one from this notebook
- Under Explore, expand this project and review the dataset and table

In [14]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=genai-demo-2024


### Review Data in BigQuery
Additional SQL queries could be used to review the data.  This section shows moving the table to a Pandas dataframe for local review in Python:
Reviewing the data helps in understanding its structure and quality before further processing.

> **Note:** <p>This query only selects one column.  This means BigQuery scans less data as it does not process the other columns.  </p>

In [15]:
# Construct a query to select a sample of the 'Class' column from the BigQuery table
query = f"""
SELECT Class
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
# Execute the query and convert the result to a pandas dataframe for easy manipulation and review in Python

df = bq.query(query = query).to_dataframe()

In [16]:
# Display the value counts of the 'Class' column
# This helps in understanding the distribution of classes in the dataset.

df['Class'].value_counts()

Class
0    276740
1       257
Name: count, dtype: Int64

In [17]:
# Display the normalized value counts of the 'Class' column
# Normalized value counts show the proportion of each class, which is useful for understanding class imbalance.

df['Class'].value_counts(normalize=True)

Class
0    0.999072
1    0.000928
Name: proportion, dtype: Float64

---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL:

In [18]:
# Construct a BigQuery client object.
client = bigquery.Client()

# Get the credentials of the current environment, which should be the service account
credentials, project = default()
request = google.auth.transport.requests.Request()
credentials.refresh(request=request)

# print the service account
print(credentials.service_account_email) 
service_account_email = credentials.service_account_email

# Construct a query to create or replace a table with a train/test/validation split
# The CASE WHEN statement splits the data into training (80%), validation (10%), and test (10%) sets.

query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits,
     "{service_account_email}" as service_account_email
FROM add_id
"""
# Execute the query to create the prepped table with the splits
job = bq.query(query = query)
job.result()

292219499736-compute@developer.gserviceaccount.com


In [19]:
# how many seconds the job took to be executed
(job.ended-job.started).total_seconds()

8.613

In [20]:
# Estimated bytes scan. To estimate the cost of the query in ON Demand pricing model
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

77.532674 MB


### Review the test/train split in BigQuery:

In [21]:
# Construct a query to review the distribution of splits
# This query counts the number of records in each split and calculates their percentage.

query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,VALIDATE,27931,10.083503
1,TEST,28153,10.163648
2,TRAIN,220913,79.752849


### Retrieve a subset of the data to a Pandas dataframe:
This allows for a quick inspection of the data in the notebook environment.

In [22]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""

# Execute the query and convert the result to a pandas dataframe
data = bq.query(query = query).to_dataframe()

In [23]:
# Display the first few rows of the dataframe
# This provides a quick view of the data to verify the contents.

data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V25,V26,V27,V28,Amount,Class,Feedback,transaction_id,splits,service_account_email
0,66341.0,-1.975549,-1.606799,0.081444,-2.159326,0.589567,0.334022,0.091121,0.745302,-1.320319,...,-0.817125,1.139974,-0.061121,-0.101225,200.5,0,very satisfied.,6a487bc0-864e-476a-89a3-711a95c3bf40,TEST,292219499736-compute@developer.gserviceaccount...
1,138794.0,1.556636,-0.858618,-1.541472,1.235769,0.142803,0.172018,0.186438,-0.093988,0.434570,...,0.427739,-0.227500,-0.026433,-0.027460,250.0,0,very satisfied.,381e5380-80f7-4b4c-b9f7-de89eda68b89,TEST,292219499736-compute@developer.gserviceaccount...
2,100922.0,1.923291,-0.511650,0.217122,0.583403,-1.043504,-0.428347,-0.946888,-0.055398,2.688932,...,-0.628523,0.441857,-0.045307,-0.039834,39.0,0,very satisfied.,90c3c8af-c5bb-4f4b-9735-a30555ba12c6,TEST,292219499736-compute@developer.gserviceaccount...
3,151295.0,-2.663683,2.410267,-0.956647,-1.124388,-0.477937,-0.984630,0.318403,0.709386,1.257919,...,0.370005,0.166635,0.867921,0.614288,20.0,0,very satisfied.,3f04990a-c791-43b6-9af6-78d786fe3ef6,TEST,292219499736-compute@developer.gserviceaccount...
4,148075.0,-0.507233,1.319840,-0.420694,-0.036181,1.742189,-0.579730,1.608304,-0.062022,-0.916780,...,0.466007,-0.652604,0.104366,0.160145,20.0,0,very satisfied.,6973debb-1be0-4142-b524-10075fe71758,TEST,292219499736-compute@developer.gserviceaccount...
